## Phase-aware speech enhancement with deep complex U-Net

Авторами статьи (Hyeong-Seok Choi1, Jang-Hyun Kim2, Jaesung Huh3, Adrian Kim3, Jung-Woo Ha3, Kyogu Lee1) предложили новый подход в задаче speech enhancement (улучшение качества речи в аудио, где на вход подается речь с шумами, а на выходе ожидается чистая речь, отделенная от шумов). 

### Несколько слов об оценке фазового сдвига

Основная сложность задачи speech enhancement заключается в оценке фазовых сдвигов чистой речи. Во ранних работах ограничивались оценкой магнитуды, а фазу брали от речи с шумами. Такой подход особенно плохо работает при высоком уровне шума (или низком snr). Поняв важность оценки фазовой составляющей, исследователи пробовали различные подходы. Например, в качестве входа модели использовали волны во временном диапазоне (без Фурье трансформации), которые уже содержат фазовую составляющую, или тренировали 2 модели для фазы и магнитуды. Также были подходы к оценке масок для фазы и магнитуды в комплексных числах, такой же как предлагают в данной статье. Однако авторы использовали MSE loss и другие "ограничительные" функции для масок, что привело к некоторым проблемам. 
Исследователи сходятся в одном, что корректная оценка фазовой составляющей в данной задаче особенно важна.

Что же было предложено авторами? Давайте разделим идеи на 3 части и поговорим о них по отдельности. 

### Архитектура сети

Unet архитектура из слоев, которые обучаются на спектрограммах в комплексных числах.
Авторы взяли классическую unet архитектуру, но состоит она теперь из convolution и deconvolution слоев, которые модифицированы для обработки вещественной и мнимой части комплексного входа. Unet архитектура известна давно и часто применяется для решения задач computer vision, и даже показала неплохие результаты в задаче speech enhancement при обучении на слоях в вещественных числах. Слои convolution и deconvolution для обработки комплексных чисел тоже были уже известны авторам. Авторы тренировали данную архитектуру для оптимизации масок (о них подробнее ниже), которые накладывались на исходную речь с шумом и через обратное преобразование Фурье преобразовывались обратно к волнам во временном диапазоне. В результате мы получаем сигнал отделенный от шумов.


### Получение масок

Маски, полученные в вещественной области, изменяют только магнитуду, но оставляют без изменения фазу. А важность оценки фазы уже обсуждалась выше. Маски в комплексной области позволяют оценить не только магнитуду, но и фазу, позволяя выполнять вращение в полярных координатах. Следующий шаг это ограничить маску, так как оптимизировать ограниченный параметр намного проще. Ранее уже были способы получить ограниченные маски для вещественной и мнимой части  выхода модели, но в прямоугольных координатах. У них обнаружилось ряд недостатков: они плохо отражают идеальные маски, так как ограничены единичным квадратом (0,0)-(1,1), а также у них ограничен угол вращения фазовой составляющей. 
Авторы статьи предложили применять маски в полярных координатах с наложением нелинейности на магнитуду (они использовали гиперболический тангенс), чтобы ограничить “магнитудную” часть маски.
Сравнивая макси, полученные авторами с идеальной маской, авторы показывают, что они довольно близко к ней приближаются.


### Loss функция

Ранее исследователи часто использовали MSE лосс для оценки качества моделей, сравнивая чистый звук и приближенный моделью. Однако были проведены исследования, которые показывают, что данный лосс плохо работает в комплексной области, плохо оптимизируя фазовую составляющую. В статье предлагается использовать новый лосс (weighted SDR), который напрямую оптимизирует метрику качества звука (source to distortion ratio). Подобный лосс был предложен в более ранних работах, но авторы его немного модифицировали. Теперь лосс это сумма двух взвешенных слагаемых: лосс для оценки полученного чистого звука и полученного шума. Оценка шума добавлена в лосс, чтобы модель обучалась только для примеров с шумом.

### Эсперименты

Авторы статьи использовали 2 открытых датасета: DEMAND и Voice Bank corpus. Для трейн части они взяли 28 голосов из Voice Bank, 8 шумов из DEMAND и 2 сгенерированных, а также 4 sound-noise-ratio (15, 10, 5, 0 (dB)). Для тестовой части датасета авторы взяли 5 шумов из DEMAND, 2 голоса из Voice Bank и 4 sound-noise-ratio (17.5, 12.5, 7.5, 2.5 (dB)). В моих экспериментах я повторила данную схему. Схема препроцессинга звуков также была повторена.

Авторы сравнивали свою модель с предыдущими подходами по нескольким метрикам, по всем показателям удались получить улучшение метрик при использовании двух самых больших по количеству параметров архитектур. 

Также они проводили эксперименты с различными архитектурами Unet (10, 16, 20 слоев), mask стратегиями и лосс функциями. Здесь в целом результаты показывают, что предложенные методы эффективно справляются с поставленной задачей, однако в некоторых случаях более ранние маски и лоссы выигрывают.  

Последний численный эксперимент авторы провели сравнивая Unet с комплексными слоями (на входе спектрограмма в комплексных числах) и Unet с вещественными слоями (2 модификации: на входе спектрограмма в комплексных числах и на входе магнитуда спектрограммы). Количество оптимизируемых параметров авторы сделали одинаковым для корректного эксперимента. Здесь результат однозначный: сеть из комплексных слоев позволяет оценивать фазовую составляющую, что является важным в данной задаче.  

Также авторы проводили качественную оценку полученных результатов “на людях”. Эксперимент показал хорошие результаты. Авторы особенно подчеркивают хорошие результаты при низких snr, с которыми предыдущие подходы плохо работают.

### Мои результаты

Мне удалось натренировать 3 архитектуры: 10, 16 и 20 (та, которая поменьше) слоев. Ниже приведены графики PESQ и WeightedSDR loss для каждой архитектуры в процессе тренировки. Светло-синей линией показаны значения для каждого трейн батча. Синей гладкой линией аппроксимирована светло синяя ломаная линия в точках, соответствующих началу эпох. Валидация моделей производилась после каждой эпохи на тест датасете, поэтому значения красных точек соответствуют PESQ и WeightedSDR loss на момент начала эпохи, отмеченной на оси Х.

### Полученные метрики в числах

| Model | DCUnet-20 | DCUnet-16 | DCUnet-10 |
|:------|-----------|-----------|-----------|
| PESQ  | table     | 3.03     | 3.0       |
| WSDR  | table     | -0.98     | -0.98      |

По моему мнению, полученные цифры подтверждают метрики, указанные в статье. Учитывая, что каждая модель тренировалась по 10 эпох, цифры хорошо коррелируют с указанными в статье

### Давайте послушаем, что получилось

#### Запись из жизни

In [9]:
# изначальная запись
ipd.Audio('../results/live_1.wav')

In [3]:
model_inference_custom(model_checkpoint_name='chp_model_32_5_epoch_9_-0.98_3.00.pth', 
                       folder_for_results='results', path_to_custom_file='../results/live_1.wav')

model_inference_custom(model_checkpoint_name='chp_model_32_10_epoch_3_-0.98_2.99.pth', 
                       folder_for_results='results', path_to_custom_file='../results/live_1.wav')

model_inference_custom(model_checkpoint_name='chp_model_32_8_epoch_6_-0.98_2.96.pth', 
                       folder_for_results='results', path_to_custom_file='../results/live_1.wav')

Init file: ../results/live_1.wav
Saved to /Users/emikhailova/Projects/Unet_speech_enhancement/results/model_32_5_9_live_1.wav inference time 0.34
Init file: ../results/live_1.wav
Saved to /Users/emikhailova/Projects/Unet_speech_enhancement/results/model_32_10_3_live_1.wav inference time 2.473
Init file: ../results/live_1.wav
Saved to /Users/emikhailova/Projects/Unet_speech_enhancement/results/model_32_8_6_live_1.wav inference time 0.495


In [12]:
# DCUnet-10
ipd.Audio('../results/model_32_5_9_live_1.wav')

In [11]:
# DCUnet-20
ipd.Audio('../results/model_32_10_3_live_1.wav')

In [10]:
# DCUnet-16
ipd.Audio('../results/model_32_8_6_live_1.wav')

### Перспективы метода

Speech enhancement - тема, у которой можно найти большое колличество приложений. Данные методы могут работать как вспомогательные для задач распознавания речи, быть частью платформ для коммуникаций (видео и аудио, но надо проверить внимательнее смотреть на время инференса), для оцифровки старых записей, а также помогать людям с ограниченным слухом. Также идеи авторов можно использовать для других задач (например взять маски или лосс функцию)